In [1]:
"""
Esse código é responsavel por tratar os dados, assim tratando os seguintes cenarios:
- Remover colunas que não serão utilizadas
- tratar linhas que possuem valores nulos
    - Colunas nulas numericas serão preenchidas com a -1
    - Colunas nulas String serão preenchidas com a string 'Desconhecido'
- tratar linhas que possuem valores duplicados
- tratar linhas que possuem valores fora do padrão
    - Colunas serão tratadas de acordo com o tipo de dado
    - float pode ser convertido para int
    - float 64 pode ser convertido para float 32 ou float 16
    - string pode ser convertida para categoria
    - int pode ser convertido para int8, int16, int32, int64
- Otimizar tipos de dados
- Salvar os tipos de dados otimizados em um arquivo externo
- Salvar os dados tratados em um novo arquivo
"""

"\nEsse código é responsavel por tratar os dados, assim tratando os seguintes cenarios:\n- Remover colunas que não serão utilizadas\n- tratar linhas que possuem valores nulos\n    - Colunas nulas numericas serão preenchidas com a -1\n    - Colunas nulas String serão preenchidas com a string 'Desconhecido'\n- tratar linhas que possuem valores duplicados\n- tratar linhas que possuem valores fora do padrão\n    - Colunas serão tratadas de acordo com o tipo de dado\n    - float pode ser convertido para int\n    - float 64 pode ser convertido para float 32 ou float 16\n    - string pode ser convertida para categoria\n    - int pode ser convertido para int8, int16, int32, int64\n- Otimizar tipos de dados\n- Salvar os tipos de dados otimizados em um arquivo externo\n- Salvar os dados tratados em um novo arquivo\n"

In [2]:
import seaborn as sns
import pandas as pd
import chardet
import json
import gc
import matplotlib.pyplot as plt

In [3]:
input_file = '../microdados_enem_2023/DADOS/MICRODADOS_ENEM_2023.csv'
output_file = 'microdados_tratado.parquet'
output_file_csv = 'microdados_tratado.csv'

In [4]:


# Verifica o encoding original do arquivo do arquivo (ISO-8859-1)
with open(input_file, 'rb') as f:
    resultado = chardet.detect(f.read(10000))
    encoding_original = resultado['encoding']

microdados_V1 = pd.read_csv(input_file, sep=';', encoding=encoding_original, low_memory=False)

In [5]:
microdados_V1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 76 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   NU_INSCRICAO            int64  
 1   NU_ANO                  int64  
 2   TP_FAIXA_ETARIA         int64  
 3   TP_SEXO                 object 
 4   TP_ESTADO_CIVIL         int64  
 5   TP_COR_RACA             int64  
 6   TP_NACIONALIDADE        int64  
 7   TP_ST_CONCLUSAO         int64  
 8   TP_ANO_CONCLUIU         int64  
 9   TP_ESCOLA               int64  
 10  TP_ENSINO               float64
 11  IN_TREINEIRO            int64  
 12  CO_MUNICIPIO_ESC        float64
 13  NO_MUNICIPIO_ESC        object 
 14  CO_UF_ESC               float64
 15  SG_UF_ESC               object 
 16  TP_DEPENDENCIA_ADM_ESC  float64
 17  TP_LOCALIZACAO_ESC      float64
 18  TP_SIT_FUNC_ESC         float64
 19  CO_MUNICIPIO_PROVA      int64  
 20  NO_MUNICIPIO_PROVA      object 
 21  CO_UF_PROVA             int64  

In [6]:
# Verifica a quantidade de linhas total do arquivo
total_linhas = microdados_V1.shape[0]
print(f'Total de linhas: {total_linhas}')

# Verifica a quantidade de linhas que possuem qualquer valor nulo
linhas_com_nulos = microdados_V1.isnull().any(axis=1).sum()
print(f'Total de linhas com algum valor nulo: {linhas_com_nulos}')

# Verifica a quantidade de linhas sem valores nulos
total_linhas_sem_nulos = total_linhas - linhas_com_nulos
print(f'Total de linhas sem valores nulos: {total_linhas_sem_nulos}')

Total de linhas: 3933955
Total de linhas com algum valor nulo: 3225341
Total de linhas sem valores nulos: 708614


In [7]:
#==================== DADOS NULOS ====================#

In [8]:
# Verifica a quantidade de linhas total do arquivo
total_linhas = microdados_V1.shape[0]
print(f'Total de linhas: {total_linhas}')

# Verifica a quantidade de linhas que possuem qualquer valor nulo
linhas_com_nulos = microdados_V1.isnull().any(axis=1).sum()
print(f'Total de linhas com algum valor nulo: {linhas_com_nulos}')

# Verifica a quantidade de linhas restantes
total_linhas_restantes = total_linhas - linhas_com_nulos
print(f'Total de linhas restantes: {total_linhas_restantes}')

Total de linhas: 3933955
Total de linhas com algum valor nulo: 3225341
Total de linhas restantes: 708614


In [9]:
# Verificar informações sobre os dados
microdados_V1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 76 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   NU_INSCRICAO            int64  
 1   NU_ANO                  int64  
 2   TP_FAIXA_ETARIA         int64  
 3   TP_SEXO                 object 
 4   TP_ESTADO_CIVIL         int64  
 5   TP_COR_RACA             int64  
 6   TP_NACIONALIDADE        int64  
 7   TP_ST_CONCLUSAO         int64  
 8   TP_ANO_CONCLUIU         int64  
 9   TP_ESCOLA               int64  
 10  TP_ENSINO               float64
 11  IN_TREINEIRO            int64  
 12  CO_MUNICIPIO_ESC        float64
 13  NO_MUNICIPIO_ESC        object 
 14  CO_UF_ESC               float64
 15  SG_UF_ESC               object 
 16  TP_DEPENDENCIA_ADM_ESC  float64
 17  TP_LOCALIZACAO_ESC      float64
 18  TP_SIT_FUNC_ESC         float64
 19  CO_MUNICIPIO_PROVA      int64  
 20  NO_MUNICIPIO_PROVA      object 
 21  CO_UF_PROVA             int64  

In [10]:
# Localizando colunas nulas
nulos = microdados_V1.columns[microdados_V1.isnull().any()]
nulos

Index(['TP_ENSINO', 'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC',
       'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC',
       'TP_SIT_FUNC_ESC', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
       'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
       'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC',
       'TX_RESPOSTAS_MT', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC',
       'TX_GABARITO_MT', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2',
       'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO'],
      dtype='object')

In [11]:
# # Localizando colunas nulas de cada tipo
# nulos_number = microdados_V1[nulos].select_dtypes(include='number').columns
# nulos_string = microdados_V1[nulos].select_dtypes(include='object').columns
# nulos_number, nulos_string

In [12]:
"""
Selecionar colunas nulas relacionadas as notas do candidato
Verificar existencia de candidatos tiraram nota 0
Verificar existencia de valores igual a -1
"""
colunas_notas = [
    'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
    'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
    'NU_NOTA_COMP5', 'NU_NOTA_REDACAO',
]

microdados_V1[colunas_notas].min() == 0

NU_NOTA_CN         True
NU_NOTA_CH         True
NU_NOTA_LC         True
NU_NOTA_MT         True
NU_NOTA_COMP1      True
NU_NOTA_COMP2      True
NU_NOTA_COMP3      True
NU_NOTA_COMP4      True
NU_NOTA_COMP5      True
NU_NOTA_REDACAO    True
dtype: bool

In [13]:
microdados_V1[colunas_notas].min() == -1

NU_NOTA_CN         False
NU_NOTA_CH         False
NU_NOTA_LC         False
NU_NOTA_MT         False
NU_NOTA_COMP1      False
NU_NOTA_COMP2      False
NU_NOTA_COMP3      False
NU_NOTA_COMP4      False
NU_NOTA_COMP5      False
NU_NOTA_REDACAO    False
dtype: bool

In [14]:
"""
Exitem candidatos que tiraram nota 0 na provas
Analisado que não existem valores -1 nas notas dos candidatos	
"""
microdados_V1[colunas_notas] = microdados_V1[colunas_notas].fillna(-1)
microdados_V1[colunas_notas].head()

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,502.0,498.9,475.6,363.2,140.0,200.0,100.0,120.0,140.0,700.0
3,459.0,508.5,507.2,466.7,140.0,200.0,160.0,180.0,200.0,880.0
4,402.5,379.2,446.9,338.3,120.0,120.0,120.0,120.0,80.0,560.0


In [15]:
# Verfica a quantidade de linhas nulas restantes nas colunas tratadas
# microdados_V1[colunas_gabarito].isna().sum().sum()
microdados_V1[colunas_notas].isna().sum().sum()

np.int64(0)

In [16]:
# Verifica a quantidade de colunas nulas restantes
colunas_com_nulos = microdados_V1.columns[microdados_V1.isnull().any()]
colunas_com_nulos

Index(['TP_ENSINO', 'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC',
       'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC',
       'TP_SIT_FUNC_ESC', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
       'CO_PROVA_MT', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC',
       'TX_RESPOSTAS_MT', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC',
       'TX_GABARITO_MT', 'TP_STATUS_REDACAO'],
      dtype='object')

In [17]:
# Extraindo colunas com valores nulos do tipo number e aplicando valores -1
colunas_com_nulos_numericos = microdados_V1[colunas_com_nulos].select_dtypes(include='number').columns


# Colunas que possuem valores nulos do tipo string, continuaram sem alteração
colunas_com_nulos_numericos

Index(['TP_ENSINO', 'CO_MUNICIPIO_ESC', 'CO_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC',
       'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'CO_PROVA_CN', 'CO_PROVA_CH',
       'CO_PROVA_LC', 'CO_PROVA_MT', 'TP_STATUS_REDACAO'],
      dtype='object')

In [18]:
microdados_V1[colunas_com_nulos_numericos] = microdados_V1[colunas_com_nulos_numericos].fillna(-1)
microdados_V1[colunas_com_nulos_numericos].head()

,TP_ENSINO,CO_MUNICIPIO_ESC,CO_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,TP_STATUS_REDACAO
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1221.0,1193.0,1204.0,1211.0,1.0
3,1.0,2304400.0,23.0,2.0,1.0,1.0,1224.0,1192.0,1202.0,1214.0,1.0
4,1.0,2311306.0,23.0,2.0,1.0,1.0,1222.0,1191.0,1201.0,1212.0,1.0


In [19]:
# Verifica a quantidade de colunas nulas restantes
# Tratar colunas nulas do tipo string com a string 'Desconhecido'
colunas_com_nulos_string = microdados_V1.columns[microdados_V1.isnull().any()]
colunas_com_nulos_string

Index(['NO_MUNICIPIO_ESC', 'SG_UF_ESC', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH',
       'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TX_GABARITO_CN',
       'TX_GABARITO_CH', 'TX_GABARITO_LC', 'TX_GABARITO_MT'],
      dtype='object')

In [20]:
microdados_V1[colunas_com_nulos_string] = microdados_V1[colunas_com_nulos_string].fillna('Desconhecido')

colunas_com_nulos_string

Index(['NO_MUNICIPIO_ESC', 'SG_UF_ESC', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH',
       'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TX_GABARITO_CN',
       'TX_GABARITO_CH', 'TX_GABARITO_LC', 'TX_GABARITO_MT'],
      dtype='object')

In [21]:
microdados_V1.columns[microdados_V1.isnull().any()]

Index([], dtype='object')

In [22]:
#==================== CRIAÇÃO DE NOVAS COLUNAS ====================#

In [23]:
# Criação da coluna NU_MEDIA_GERAL

microdados_V1['NU_MEDIA_GERAL'] = microdados_V1[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
                                            'NU_NOTA_MT', 'NU_NOTA_REDACAO']].mean(axis=1).astype('float16')

In [24]:
# Criação da coluna REGIAO

# Mapeamento de estados para regiões brasileiras
ESTADO_PARA_REGIAO = {
    # Norte
    'AC': 'Norte', 'AM': 'Norte', 'AP': 'Norte', 'PA': 'Norte', 
    'RO': 'Norte', 'RR': 'Norte', 'TO': 'Norte',
    
    # Nordeste  
    'AL': 'Nordeste', 'BA': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste',
    'PB': 'Nordeste', 'PE': 'Nordeste', 'PI': 'Nordeste', 'RN': 'Nordeste', 'SE': 'Nordeste',
    
    # Centro-Oeste
    'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'MS': 'Centro-Oeste', 'MT': 'Centro-Oeste',
    
    # Sudeste
    'ES': 'Sudeste', 'MG': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
    
    # Sul
    'PR': 'Sul', 'RS': 'Sul', 'SC': 'Sul'
}

def adicionar_coluna_regiao(microdados_V3):
    """
    Adiciona a coluna REGIAO ao DataFrame baseada na coluna SG_UF_PROVA.
    Esta coluna estava sendo criada em runtime e impactava a performance.
    """
    if 'SG_REGIAO' in microdados_V3.columns:
        print("Coluna REGIAO já existe no DataFrame")
        return microdados_V3
    
    if 'SG_UF_PROVA' not in microdados_V3.columns:
        print("AVISO: Coluna SG_UF_PROVA não encontrada. Coluna REGIAO não foi criada.")
        return microdados_V3
    
    print("Criando coluna REGIAO...")
    microdados_V3['SG_REGIAO'] = microdados_V3['SG_UF_PROVA'].map(ESTADO_PARA_REGIAO)
    
    # Verificar valores não mapeados
    valores_nao_mapeados = microdados_V3[microdados_V3['SG_REGIAO'].isna()]['SG_UF_PROVA'].unique()
    if len(valores_nao_mapeados) > 0:
        print(f"AVISO: Estados não mapeados encontrados: {valores_nao_mapeados}")
        microdados_V3['SG_REGIAO'] = microdados_V3['SG_REGIAO'].fillna('Indefinido')
    
    # Otimizar tipo de dados
    microdados_V3['SG_REGIAO'] = microdados_V3['SG_REGIAO'].astype('category')
    
    print(f"Coluna REGIAO criada com {microdados_V3['SG_REGIAO'].nunique()} regiões únicas:")
    print(microdados_V3['SG_REGIAO'].value_counts())
    
    return 

# Adicionar a coluna REGIAO ao DataFrame
adicionar_coluna_regiao(microdados_V1)

Criando coluna REGIAO...
Coluna REGIAO criada com 5 regiões únicas:
SG_REGIAO
Nordeste        1423999
Sudeste         1305362
Norte            453454
Sul              417688
Centro-Oeste     333452
Name: count, dtype: int64


In [25]:
# Criação da coluna NU_DESEMPENHO"

# Calculando a média das notas de redação e competências
microdados_V1['NU_DESEMPENHO'] = microdados_V1[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
                                            'NU_NOTA_MT', 'NU_NOTA_REDACAO']].mean(axis=1)

# Função para mapear os valores
def map_desempenho(valor):
    if valor > 600:    # ~20-25% dos estudantes
        return 1        # Alto
    elif valor >= 300:  # ~30-35% dos estudantes
        return 2        # Médio
    else:               # ~40-45% dos estudantes
        return 3        # Baixo

# # Aplicando a função à coluna 'NU_DESEMPENHO'
microdados_V1['NU_DESEMPENHO'] = microdados_V1['NU_DESEMPENHO'].apply(map_desempenho).astype('category')

In [26]:
# Criação da coluna TP_PRESENCA_REDACAO

# Linguagens e Códigos (LC) E Ciências Humanas (CH)
condicao_presente_redacao = (
    (microdados_V1['TP_PRESENCA_LC'] == 1) & 
    (microdados_V1['TP_PRESENCA_CH'] == 1) 
)

# Criando a coluna com valores 1 (presente) ou 0 (ausente)
microdados_V1['TP_PRESENCA_REDACAO'] = condicao_presente_redacao.astype(int).astype('category')

# Verificação rápida
print("Distribuição de presença na redação:")
print(microdados_V1['TP_PRESENCA_REDACAO'].value_counts())

Distribuição de presença na redação:
TP_PRESENCA_REDACAO
1    2822643
0    1111312
Name: count, dtype: int64


In [27]:
# Criação da coluna TP_PRESENCA_GERAL
 
# Definindo as condições para cada categoria de presença geral
condicao_presenca_completa = (
    (microdados_V1['TP_PRESENCA_CN'] == 1) & 
    (microdados_V1['TP_PRESENCA_CH'] == 1) & 
    (microdados_V1['TP_PRESENCA_LC'] == 1) & 
    (microdados_V1['TP_PRESENCA_MT'] == 1)
)

condicao_presenca_primeiro_dia = (
    (microdados_V1['TP_PRESENCA_LC'] == 1) & 
    (microdados_V1['TP_PRESENCA_CH'] == 1) & 
    ((microdados_V1['TP_PRESENCA_MT'] != 1) | (microdados_V1['TP_PRESENCA_CN'] != 1))
)

condicao_presenca_segundo_dia = (
    (microdados_V1['TP_PRESENCA_MT'] == 1) & 
    (microdados_V1['TP_PRESENCA_CN'] == 1) & 
    ((microdados_V1['TP_PRESENCA_LC'] != 1) | (microdados_V1['TP_PRESENCA_CH'] != 1))
)

# Criando a coluna com valores padrão (0)
microdados_V1['TP_PRESENCA_GERAL'] = 0

# Aplicando as condições na ordem correta
microdados_V1.loc[condicao_presenca_primeiro_dia, 'TP_PRESENCA_GERAL'] = 1
microdados_V1.loc[condicao_presenca_segundo_dia, 'TP_PRESENCA_GERAL'] = 2
microdados_V1.loc[condicao_presenca_completa, 'TP_PRESENCA_GERAL'] = 3

# Convertendo para category
microdados_V1['TP_PRESENCA_GERAL'] = microdados_V1['TP_PRESENCA_GERAL'].astype('category')

In [28]:
# Criação da coluna NU_INFRAESTRUTURA

# Dicionário original de pontuações brutas para Q007–Q025
pontuacao_bruta = {
    'Q007': {'A': 0, 'B': 1, 'C': 2, 'D': 3},
    'Q008': {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4},
    'Q009': {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4},
    'Q010': {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4},
    'Q011': {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4},
    'Q012': {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4},
    'Q013': {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4},
    'Q014': {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4},
    'Q015': {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4},
    'Q016': {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4},
    'Q017': {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4},
    'Q018': {'A': 0, 'B': 1},
    'Q019': {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4},
    'Q020': {'A': 0, 'B': 1},
    'Q021': {'A': 0, 'B': 1},
    'Q022': {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4},
    'Q023': {'A': 0, 'B': 1},
    'Q024': {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4},
    'Q025': {'A': 0, 'B': 1},
}

# Lista de colunas que entram no índice
colunas_infra = list(pontuacao_bruta.keys())


# === 2️⃣ Mapeamento e Normalização ===

for col in colunas_infra:
    mapping = pontuacao_bruta[col]
    max_raw = max(mapping.values())
    # cria coluna normalizada de 0.0 a 1.0
    microdados_V1[col + "_norm"] = (
        microdados_V1[col]
        .map(mapping)           # troca letra por valor bruto
        .fillna(0)              # trata missings como zero
        .div(max_raw)           # normaliza dividindo pelo máximo
    )


# === 3️⃣ Cálculo do Índice de Infraestrutura ===

# média simples dos componentes normalizados (escala 0–1)
norm_cols = [c + "_norm" for c in colunas_infra]
microdados_V1["infra_norm_media"] = microdados_V1[norm_cols].mean(axis=1)


# === 4️⃣ Definição das Três Categorias ===

q1 = microdados_V1["infra_norm_media"].quantile(0.33)
q2 = microdados_V1["infra_norm_media"].quantile(0.66)

def categorizar(x):
    if x >= q2:
        return "Alta"
    elif x >= q1:
        return "Média"
    else:
        return "Baixa"

microdados_V1["NU_INFRAESTRUTURA"] = (
    microdados_V1["infra_norm_media"]
    .apply(categorizar)
    .astype("category")
)


# === 5️⃣ Limpeza Final ===

# remove colunas temporárias de normalização
microdados_V1.drop(columns=norm_cols + ["infra_norm_media"], inplace=True)


In [29]:
# Criação da coluna TP_FAIXA_SALARIAL

# Criar mapeamento de faixa salarial para número de salários mínimos
salario_mapping = {
    'A': 0,            # Nenhuma Renda
    'B': 1,            # Até 1 salário mínimo
    'C': 2,            # 1-1.5 salários mínimos
    'D': 2,            # 1.5-2 salários mínimos
    'E': 3,            # 2-2.5 salários mínimos
    'F': 3,            # 2.5-3 salários mínimos
    'G': 4,            # 3-4 salários mínimos
    'H': 4,            # 4-5 salários mínimos
    'I': 5,            # 5-6 salários mínimos
    'J': 5,            # 6-7 salários mínimos
    'K': 5,            # 7-8 salários mínimos
    'L': 5,            # 8-9 salários mínimos
    'M': 5,            # 9-10 salários mínimos
    'N': 6,            # 10-12 salários mínimos
    'O': 6,            # 12-15 salários mínimos
    'P': 6,            # 15-20 salários mínimos
    'Q': 7             # Acima de 20 salários mínimos
}

# Descrição das categorias
faixa_salarial_descricao = {
    0: 'Nenhuma Renda',
    1: 'Até 1 salário mínimo',
    2: '1 a 2 salários mínimos',
    3: '2 a 3 salários mínimos',
    4: '3 a 5 salários mínimos',
    5: '5 a 10 salários mínimos',
    6: '10 a 20 salários mínimos',
    7: 'Acima de 20 salários mínimos'
}

# Aplicar o mapeamento e criar a nova coluna
microdados_V1['TP_FAIXA_SALARIAL'] = microdados_V1['Q006'].map(salario_mapping).astype('category')

# Verificar a distribuição
print("Distribuição da nova faixa salarial:")
print(microdados_V1['TP_FAIXA_SALARIAL'].value_counts().sort_index())

Distribuição da nova faixa salarial:
TP_FAIXA_SALARIAL
0     268053
1    1245271
2    1088308
3     465338
4     400606
5     294002
6     120888
7      51489
Name: count, dtype: int64


In [30]:
#==================== OTIMIZAÇÂO DE DTYPES ====================#

In [31]:
microdados_V2 = microdados_V1.copy()
del microdados_V1
gc.collect()

20

In [32]:
microdados_V2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 83 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   NU_INSCRICAO            int64   
 1   NU_ANO                  int64   
 2   TP_FAIXA_ETARIA         int64   
 3   TP_SEXO                 object  
 4   TP_ESTADO_CIVIL         int64   
 5   TP_COR_RACA             int64   
 6   TP_NACIONALIDADE        int64   
 7   TP_ST_CONCLUSAO         int64   
 8   TP_ANO_CONCLUIU         int64   
 9   TP_ESCOLA               int64   
 10  TP_ENSINO               float64 
 11  IN_TREINEIRO            int64   
 12  CO_MUNICIPIO_ESC        float64 
 13  NO_MUNICIPIO_ESC        object  
 14  CO_UF_ESC               float64 
 15  SG_UF_ESC               object  
 16  TP_DEPENDENCIA_ADM_ESC  float64 
 17  TP_LOCALIZACAO_ESC      float64 
 18  TP_SIT_FUNC_ESC         float64 
 19  CO_MUNICIPIO_PROVA      int64   
 20  NO_MUNICIPIO_PROVA      object  
 21  CO_UF_PR

In [33]:
#Selecionar as colunas que podem ser convertidas para category
colunas_categoria = [
    "TP_FAIXA_ETARIA", "TP_SEXO", "TP_ESTADO_CIVIL", "TP_COR_RACA", "TP_NACIONALIDADE",
    "TP_ST_CONCLUSAO", "TP_ANO_CONCLUIU", "TP_ESCOLA", "TP_DEPENDENCIA_ADM_ESC", "TP_ENSINO", "IN_TREINEIRO", "NO_MUNICIPIO_ESC",
    "SG_UF_ESC", "TP_LOCALIZACAO_ESC", "TP_SIT_FUNC_ESC", "NO_MUNICIPIO_PROVA", "SG_UF_PROVA", "TP_PRESENCA_CN",
    "TP_PRESENCA_CH", "TP_PRESENCA_LC", "TP_PRESENCA_MT", "TP_LINGUA", "TP_STATUS_REDACAO",
    "Q001", "Q002", "Q003", "Q004", "Q005", "Q006", "Q007", "Q008", "Q009", "Q010", "Q011", "Q012",
    "Q013", "Q014", "Q015", "Q016", "Q017", "Q018", "Q019", "Q020", "Q021", "Q022", "Q023",
    "Q024", "Q025", 'SG_REGIAO', 'NU_DESEMPENHO', 'TP_PRESENCA_REDACAO', 'TP_PRESENCA_GERAL', 'NU_INFRAESTRUTURA', 'TP_FAIXA_SALARIAL'
]
microdados_V2[colunas_categoria] = microdados_V2[colunas_categoria].astype('category')

In [34]:
# Separar colunas por tipo de dado
print('Colunas do tipo int64:')
colunas_int64 = microdados_V2.select_dtypes(include='int64').columns
print(colunas_int64)

print('\nColunas do tipo float64:')
colunas_float64 = microdados_V2.select_dtypes(include='float64').columns
print(colunas_float64)

Colunas do tipo int64:
Index(['NU_INSCRICAO', 'NU_ANO', 'CO_MUNICIPIO_PROVA', 'CO_UF_PROVA'], dtype='object')

Colunas do tipo float64:
Index(['CO_MUNICIPIO_ESC', 'CO_UF_ESC', 'CO_PROVA_CN', 'CO_PROVA_CH',
       'CO_PROVA_LC', 'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
       'NU_NOTA_MT', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3',
       'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO'],
      dtype='object')


In [35]:
"""
Verificar se é possivel converter os valores do tipo float64 para int64
Verificar se todas as colunas do tipo float64 possuem valores inteiros
"""

resultados = {}
for coluna in colunas_float64:
    resultados[coluna] = microdados_V2[coluna].apply(lambda x: x.is_integer()).all()

for coluna, resultado in resultados.items():
    print(f'Colunas: {coluna} - Possui apenas valores inteiros: {resultado}')

Colunas: CO_MUNICIPIO_ESC - Possui apenas valores inteiros: True
Colunas: CO_UF_ESC - Possui apenas valores inteiros: True
Colunas: CO_PROVA_CN - Possui apenas valores inteiros: True
Colunas: CO_PROVA_CH - Possui apenas valores inteiros: True
Colunas: CO_PROVA_LC - Possui apenas valores inteiros: True
Colunas: CO_PROVA_MT - Possui apenas valores inteiros: True
Colunas: NU_NOTA_CN - Possui apenas valores inteiros: False
Colunas: NU_NOTA_CH - Possui apenas valores inteiros: False
Colunas: NU_NOTA_LC - Possui apenas valores inteiros: False
Colunas: NU_NOTA_MT - Possui apenas valores inteiros: False
Colunas: NU_NOTA_COMP1 - Possui apenas valores inteiros: True
Colunas: NU_NOTA_COMP2 - Possui apenas valores inteiros: True
Colunas: NU_NOTA_COMP3 - Possui apenas valores inteiros: True
Colunas: NU_NOTA_COMP4 - Possui apenas valores inteiros: True
Colunas: NU_NOTA_COMP5 - Possui apenas valores inteiros: True
Colunas: NU_NOTA_REDACAO - Possui apenas valores inteiros: True


In [36]:
"""
Selecionar colunas que possuem apenas valores float
Analizar valor maximo para verificar se é possivel converter para float16
Converter colunas float64 para float16
"""
colunas_float16 = [
    'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_MEDIA_GERAL'
]

print(f'Valor máximo encontrado: {microdados_V2[colunas_float16].max().max()}')
microdados_V2[colunas_float16] = microdados_V2[colunas_float16].astype('float16')
microdados_V2.info()

Valor máximo encontrado: 958.6
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 83 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   NU_INSCRICAO            int64   
 1   NU_ANO                  int64   
 2   TP_FAIXA_ETARIA         category
 3   TP_SEXO                 category
 4   TP_ESTADO_CIVIL         category
 5   TP_COR_RACA             category
 6   TP_NACIONALIDADE        category
 7   TP_ST_CONCLUSAO         category
 8   TP_ANO_CONCLUIU         category
 9   TP_ESCOLA               category
 10  TP_ENSINO               category
 11  IN_TREINEIRO            category
 12  CO_MUNICIPIO_ESC        float64 
 13  NO_MUNICIPIO_ESC        category
 14  CO_UF_ESC               float64 
 15  SG_UF_ESC               category
 16  TP_DEPENDENCIA_ADM_ESC  category
 17  TP_LOCALIZACAO_ESC      category
 18  TP_SIT_FUNC_ESC         category
 19  CO_MUNICIPIO_PROVA      int64   
 20  NO_MUNICIPIO_PR

In [37]:
"""
Restante das colunas podem ser convertidas para int64 inicialmente e depois tratada conforme a necessidade
"""

colunas_float64=microdados_V2.select_dtypes(include='float64').columns
colunas_float64


Index(['CO_MUNICIPIO_ESC', 'CO_UF_ESC', 'CO_PROVA_CN', 'CO_PROVA_CH',
       'CO_PROVA_LC', 'CO_PROVA_MT', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2',
       'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO'],
      dtype='object')

In [38]:
microdados_V2[colunas_float64] = microdados_V2[colunas_float64].astype('int64')

In [39]:
"""
Seleciona as colunas que possuem valores int64
"""
colunas_int64 = microdados_V2.select_dtypes(include='int64').columns
colunas_int64

Index(['NU_INSCRICAO', 'NU_ANO', 'CO_MUNICIPIO_ESC', 'CO_UF_ESC',
       'CO_MUNICIPIO_PROVA', 'CO_UF_PROVA', 'CO_PROVA_CN', 'CO_PROVA_CH',
       'CO_PROVA_LC', 'CO_PROVA_MT', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2',
       'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO'],
      dtype='object')

In [40]:
"""
Converter colunas do tipo string seu respectivo tipo de acordo com seu tamanho 
"""
colunas_int64 = microdados_V2.select_dtypes(include='int64').columns

for col in colunas_int64:
  min_val = microdados_V2[col].min()
  max_val = microdados_V2[col].max()

  if -128 <= min_val <= max_val <= 127:
      microdados_V2[col] = microdados_V2[col].astype('int8')
  elif -32768 <= min_val <= max_val <= 32767:
      microdados_V2[col] = microdados_V2[col].astype('int16')
  elif -2147483648 <= min_val <= max_val <= 2147483647:
      microdados_V2[col] = microdados_V2[col].astype('int32')
  else:
      microdados_V2[col] = microdados_V2[col].astype('int64')

microdados_V2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 83 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   NU_INSCRICAO            int64   
 1   NU_ANO                  int16   
 2   TP_FAIXA_ETARIA         category
 3   TP_SEXO                 category
 4   TP_ESTADO_CIVIL         category
 5   TP_COR_RACA             category
 6   TP_NACIONALIDADE        category
 7   TP_ST_CONCLUSAO         category
 8   TP_ANO_CONCLUIU         category
 9   TP_ESCOLA               category
 10  TP_ENSINO               category
 11  IN_TREINEIRO            category
 12  CO_MUNICIPIO_ESC        int32   
 13  NO_MUNICIPIO_ESC        category
 14  CO_UF_ESC               int8    
 15  SG_UF_ESC               category
 16  TP_DEPENDENCIA_ADM_ESC  category
 17  TP_LOCALIZACAO_ESC      category
 18  TP_SIT_FUNC_ESC         category
 19  CO_MUNICIPIO_PROVA      int32   
 20  NO_MUNICIPIO_PROVA      category
 21  CO_UF_PR

In [41]:
# Extrair colunas do tipo object
colunas_object =  microdados_V2.select_dtypes(include=['object']).columns
print(colunas_object)

Index(['TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC',
       'TX_RESPOSTAS_MT', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC',
       'TX_GABARITO_MT'],
      dtype='object')


In [42]:
colunas_object = microdados_V2.select_dtypes(include=['object']).columns
colunas_object

Index(['TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC',
       'TX_RESPOSTAS_MT', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC',
       'TX_GABARITO_MT'],
      dtype='object')

In [43]:
# Converter colunas restantes do tipo object para string
colunas_object = microdados_V2.select_dtypes(include=['object']).columns
microdados_V2[colunas_object] = microdados_V2[colunas_object].astype('string')
microdados_V2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 83 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   NU_INSCRICAO            int64   
 1   NU_ANO                  int16   
 2   TP_FAIXA_ETARIA         category
 3   TP_SEXO                 category
 4   TP_ESTADO_CIVIL         category
 5   TP_COR_RACA             category
 6   TP_NACIONALIDADE        category
 7   TP_ST_CONCLUSAO         category
 8   TP_ANO_CONCLUIU         category
 9   TP_ESCOLA               category
 10  TP_ENSINO               category
 11  IN_TREINEIRO            category
 12  CO_MUNICIPIO_ESC        int32   
 13  NO_MUNICIPIO_ESC        category
 14  CO_UF_ESC               int8    
 15  SG_UF_ESC               category
 16  TP_DEPENDENCIA_ADM_ESC  category
 17  TP_LOCALIZACAO_ESC      category
 18  TP_SIT_FUNC_ESC         category
 19  CO_MUNICIPIO_PROVA      int32   
 20  NO_MUNICIPIO_PROVA      category
 21  CO_UF_PR

In [44]:
#==================== SALVAR DADOS TRATADOS ====================#

In [45]:
microdados_V3 = microdados_V2.copy()

In [46]:
#Remover colunas que não sera usadas
microdados_V3.drop(columns=[
    'NU_INSCRICAO', 'NU_ANO', 'TP_NACIONALIDADE', 'TP_ANO_CONCLUIU', 'TP_ESCOLA',
    'TP_ENSINO', 'IN_TREINEIRO', 'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC',
    'SG_UF_ESC', 'TP_SIT_FUNC_ESC', 'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA',
    'CO_UF_PROVA', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT',
    'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT',
    'TP_LINGUA', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC',
    'TX_GABARITO_MT', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2',
    'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'Q003', 'Q004', 'Q007',
    'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015', 'Q016',
    'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024'
], inplace=True)

In [47]:
microdados_V3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 29 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   TP_FAIXA_ETARIA         category
 1   TP_SEXO                 category
 2   TP_ESTADO_CIVIL         category
 3   TP_COR_RACA             category
 4   TP_ST_CONCLUSAO         category
 5   TP_DEPENDENCIA_ADM_ESC  category
 6   TP_LOCALIZACAO_ESC      category
 7   SG_UF_PROVA             category
 8   TP_PRESENCA_CN          category
 9   TP_PRESENCA_CH          category
 10  TP_PRESENCA_LC          category
 11  TP_PRESENCA_MT          category
 12  NU_NOTA_CN              float16 
 13  NU_NOTA_CH              float16 
 14  NU_NOTA_LC              float16 
 15  NU_NOTA_MT              float16 
 16  NU_NOTA_REDACAO         int16   
 17  Q001                    category
 18  Q002                    category
 19  Q005                    category
 20  Q006                    category
 21  Q025    

In [48]:
# Mapeia os tipos de dados otimizados
dtypes_dict = microdados_V3.dtypes.apply(str).to_dict()

# Salva o mapeamento em um arquivo JSON
with open('dtypes.json', 'w') as f:
    json.dump(dtypes_dict, f)

In [49]:
# Salva o conjunto de dados tratado em um novo arquivo CSV
microdados_V3.to_parquet(output_file, index=False, engine='pyarrow')
# microdados_V3.to_csv(output_file_csv, index=False, sep=';', encoding='ISO-8859-1')

In [50]:
microdados_V3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 29 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   TP_FAIXA_ETARIA         category
 1   TP_SEXO                 category
 2   TP_ESTADO_CIVIL         category
 3   TP_COR_RACA             category
 4   TP_ST_CONCLUSAO         category
 5   TP_DEPENDENCIA_ADM_ESC  category
 6   TP_LOCALIZACAO_ESC      category
 7   SG_UF_PROVA             category
 8   TP_PRESENCA_CN          category
 9   TP_PRESENCA_CH          category
 10  TP_PRESENCA_LC          category
 11  TP_PRESENCA_MT          category
 12  NU_NOTA_CN              float16 
 13  NU_NOTA_CH              float16 
 14  NU_NOTA_LC              float16 
 15  NU_NOTA_MT              float16 
 16  NU_NOTA_REDACAO         int16   
 17  Q001                    category
 18  Q002                    category
 19  Q005                    category
 20  Q006                    category
 21  Q025    